## Day 6: Guard Gallivant

### Part One

The Historians use their fancy device again, this time to whisk you all away to the North Pole prototype suit manufacturing lab... in the year 1518! It turns out that having direct access to history is very convenient for a group of historians.

You still have to be careful of time paradoxes, and so it will be important to avoid anyone from 1518 while The Historians search for the Chief. Unfortunately, a single guard is patrolling this part of the lab.

Maybe you can work out where the guard will go ahead of time so that The Historians can search safely?

You start by making a map (your puzzle input) of the situation. For example:

```
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
```

The map shows the current position of the guard with ^ (to indicate the guard is currently facing up from the perspective of the map). Any obstructions - crates, desks, alchemical reactors, etc. - are shown as #.

Lab guards in 1518 follow a very strict patrol protocol which involves repeatedly following these steps:

* If there is something directly in front of you, turn right 90 degrees.
* Otherwise, take a step forward.

Following the above protocol, the guard moves up several times until she reaches an obstacle (in this case, a pile of failed suit prototypes):

```
....#.....
....^....#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#...
```

Because there is now an obstacle in front of the guard, she turns right before continuing straight in her new facing direction:

```
....#.....
........>#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#...
```

Reaching another obstacle (a spool of several very long polymers), she turns right again and continues downward:

```
....#.....
.........#
..........
..#.......
.......#..
..........
.#......v.
........#.
#.........
......#...
```

This process continues for a while, but the guard eventually leaves the mapped area (after walking past a tank of universal solvent):

```
....#.....
.........#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#v..
```

By predicting the guard's route, you can determine which specific positions in the lab will be in the patrol path. Including the guard's starting position, the positions visited by the guard before leaving the area are marked with an X:

```
....#.....
....XXXXX#
....X...X.
..#.X...X.
..XXXXX#X.
..X.X.X.X.
.#XXXXXXX.
.XXXXXXX#.
#XXXXXXX..
......#X..
```

In this example, the guard will visit `41` distinct positions on your map.

Predict the path of the guard. How many distinct positions will the guard visit before leaving the mapped area?

### Part One Plan

An object will represent a guard that tracks their position, direction, and which positions the guard has visited.

In [1]:
const input = true ?
	await Deno.readTextFile("./work/2024/06/day6input.txt") :
	`
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...`.trim()
const rows = input.split("\n")

In [2]:
// The directions are in order of turn direction.
enum Direction {
	North = [-1, 0],
	East = [0, 1],
	South = [1, 0],
	West = [0, -1],
}
type Position = [int, int]
type Guard = {
	direction: Direction,
	position: Position,
	visited: string[], // copy of the map
	turn: () => void
}
function moveDirection(p:Position, d:Direction): Position {
	let [pr, pc] = p
	let [vr, vc] = d
	return [pr+vr, pc+vc]
}
// Find the initial position of the guard. Assume the direction is north.
let initialPosition:Position = rows.reduce(function(init:Position|null, row:string, rowi:int): Position {
	const coli = row.indexOf("^")
	if (coli > -1) {
		return [rowi, coli]
	}
	return init
}, null)

const guard:Guard = {
	direction: Direction.North,
	position: initialPosition,
	visited: [...rows],
	markVisited: () => {
		const row = guard.visited[guard.position[0]].split('')
		row[guard.position[1]] = 'X'
		guard.visited[guard.position[0]] = row.join('')
	},
	turn: () => {
		const directions:Direction[] = Object.values(Direction)
		// Adding two to the  index, because Object.values gives the Position and then the enum label
		guard.direction = directions[(directions.indexOf(guard.direction) + 2) % directions.length]
	}
}
while (true) {
	guard.markVisited()
	const [nrow, ncol]:Position = moveDirection(guard.position, guard.direction)
	if (rows[nrow]?.charAt(ncol) == '#') {
		guard.turn()
		continue
	} else if (['.', '^', 'X'].includes(rows[nrow]?.charAt(ncol))) {
		guard.position = [nrow, ncol]
		continue
	}
	break
}
const visited = guard.visited.reduce((sum:int, row:string) => {
	return sum + (row.match(/X/g) || '').length
}, 0)
console.log(`Distinct positions visited: ${visited}`)

Distinct positions visited: 4967


### Part Two

While The Historians begin working around the guard's patrol route, you borrow their fancy device and step outside the lab. From the safety of a supply closet, you time travel through the last few months and record the nightly status of the lab's guard post on the walls of the closet.

Returning after what seems like only a few seconds to The Historians, they explain that the guard's patrol area is simply too large for them to safely search the lab without getting caught.

Fortunately, they are pretty sure that adding a single new obstruction won't cause a time paradox. They'd like to place the new obstruction in such a way that the guard will get stuck in a loop, making the rest of the lab safe to search.

To have the lowest chance of creating a time paradox, The Historians would like to know all of the possible positions for such an obstruction. The new obstruction can't be placed at the guard's starting position - the guard is there right now and would notice.

In the above example, there are only `6` different positions where a new obstruction would cause the guard to get stuck in a loop. The diagrams of these six situations use `O` to mark the new obstruction, `|` to show a position where the guard moves up/down, `-` to show a position where the guard moves left/right, and `+` to show a position where the guard moves both up/down and left/right.

Option one, put a printing press next to the guard's starting position:

```
....#.....
....+---+#
....|...|.
..#.|...|.
....|..#|.
....|...|.
.#.O^---+.
........#.
#.........
......#...
```

Option two, put a stack of failed suit prototypes in the bottom right quadrant of the mapped area:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
......O.#.
#.........
......#...
```

Option three, put a crate of chimney-squeeze prototype fabric next to the standing desk in the bottom right quadrant:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
.+----+O#.
#+----+...
......#...
```

Option four, put an alchemical retroencabulator near the bottom left corner:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
..|...|.#.
#O+---+...
......#...
```

Option five, put the alchemical retroencabulator a bit to the right instead:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
....|.|.#.
#..O+-+...
......#...
```

Option six, put a tank of sovereign glue right next to the tank of universal solvent:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
.+----++#.
#+----++..
......#O..
```

It doesn't really matter what you choose to use as an obstacle so long as you and The Historians can put it into position without the guard noticing. The important thing is having enough options that you can find one that minimizes time paradoxes, and in this example, there are 6 different positions you could choose.

You need to get the guard stuck in a loop by adding a single new obstruction. How many different positions could you choose for this obstruction?


### Part Two Plan

How to determine a loop has occurred? Is it if the guard crosses his starting position once or twice? Another possibility is to see if the guard encounters our block twice?

> Retro: Instead of looking for the obstruction (because the guard may not revisit it in some situations), I'm tracking how many times the guard has been to a position. The maximum he can visit it is 4 times, which means it's a loop. Not the most efficient...

So it would try until the guard encounters our block twice or leaves the loop, then we try another obstruction location and iterate through (Brute force). Comparing all the difference loops in the example above with the examples filled in map, all of the obstacles exist within the `X` marked spaces. This makes sense because the guard can't encounter an obstruction outside of the path he initially travels.

The previous solution I marked the map with `X` on all places the guard visits, so I can iterate through those to place obstructions (except for the starting position).



In [3]:
let visitedCopy = [...guard.visited]
const loopsFound = visitedCopy.reduce((sum:int, row:string, rowi:int) => {
	row.split('').forEach((c:char, coli:int) => {
		if (rowi == initialPosition[0] && coli == initialPosition[1]) {
			return
		}
		if (c == 'X') {
			// Make a copy of the original
			const mapCopy = [...rows]
			// Set the obstical
			const orow = mapCopy[rowi].split('')
			orow[coli] = 'O'
			mapCopy[rowi] = orow.join('')
			// Reset the guard
			guard.direction = Direction.North
			guard.position = initialPosition
			while (true) {
				const [nrow, ncol]: Position = moveDirection(guard.position, guard.direction)
				const symbol: char = mapCopy[nrow]?.charAt(ncol)
				if (symbol?.match(/[#O]/)) {
					guard.turn()
					continue
				} else if (symbol?.match(/[\.\^1-4]/)) {
					guard.position = [nrow, ncol]
					let mapRow = mapCopy[nrow].split('')
					mapRow[ncol] = symbol.match(/[^1-4]/) ? 1 : parseInt(symbol, 10) + 1
					mapCopy[nrow] = mapRow.join('')
					if (mapRow[ncol] >= 4) {
						sum++
						break
					}
					continue
				}
				break
			}
		}
	})
	return sum
}, 0)
console.log(`Loop positions: ${loopsFound}`)

Loop positions: 1795


This part two solution doesn't quite produce the correct answer, but I'm moving on for now.